In [47]:
import librosa
import matplotlib.pyplot as plt
import numpy as np
import IPython.display as ipd
import os
import random
import wave
from array import array
import soundfile as sf
import sounddevice as sd
import scipy.io.wavfile as ww
from scipy.io.wavfile import read
import glob

In [48]:
PATH_NOISE= os.getcwd()+'/word/noise'
WORD_TO_TRAIN = ['cipi','bangun']
PATH_TO_TRAIN = './DeadSimpleSpeechRecognizer/newdata/train'
PATH_TO_TEST = './word/test'
PATH_TO_DEV = './word/dev'


#PATH_TO_TRAIN = os.getcwd()+'/data/train'
#PATH_TO_TEST = os.getcwd()+'/data/test'
FILE_PREFIX = "ns"
WORKPATH = PATH_TO_TRAIN


chunk = 2048
CHANNELS = 1
RATE = 16000
length_capture = 20  #20 ms
overlap_frame = 4    #4 ms
start_at = 7         #maybe not used
end_at = 100
DEBUG = False

In [49]:
def getlistwav(path, dir=""):
    if dir == "": dir = path + "/"
    tmp = os.listdir(path)
    result = []
    for i in tmp:
        if str(i).find('wav',0)>0: result.append(dir+i)
    return result

In [50]:
def writefile(filename, data):
    p = pyaudio.PyAudio()
    wf = wave.open(filename, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(data)
    wf.close() 

def addblank(filename, front=0, back=0):
    wv, sr = sf.read(filename)
    wv=np.pad(wv,(front,back),'constant')
    plt.plot(wv)
    return wv

def getlen(filename):
    wv, sr = sf.read(filename)
    print('Length:',len(wv))
    return len(wv)

def pitch(filename,target=None, n_steps=0.0):
    wv,sr = sf.read(filename)
    print(wv)
    wv = wv.T
    wv = librosa.resample(wv,sr,44100,scale=True)
    y_ = librosa.effects.pitch_shift(wv, 44100, n_steps=n_steps)
    y_ = librosa.resample(y_,44100,sr,scale=True)
    #print(y_)
    sd.play(y_,samplerate=sr)
    if target!=None: sf.write(target,y_,sr)

def add_random_noise(filename, target):

    #add some blanks front and back
    front = random.randint(0,5000)
    back = random.randint(0,5000)

    wv, sr = sf.read(filename)
    wv=np.pad(wv,(front,back),'constant')


    #random pitch
    ptch = random.uniform(-1,1)

    wv = wv.T
    wv = librosa.resample(wv,sr,44100,scale=True)
    wv = librosa.effects.pitch_shift(wv, 44100, n_steps=ptch)
    #y_ = librosa.resample(y_,44100,sr,scale=True)
    #print(y_)

    #get the noise
    lnoise = getlistwav(PATH_NOISE, dir="",)
    random.shuffle(lnoise)
    wv2, sr2 = sf.read(lnoise[0])
    

    wv2 = wv2.T
    wv2 = librosa.resample(wv2,sr2,44100,scale=True)
    
    rv = random.randint(0,len(wv2)-len(wv))
    wv2 = wv2[rv:rv+len(wv),]
    #wv3 = ((wv * random.uniform(0.9,1.3)) + (wv2 * random.uniform(0,0.4))) / 2
    wv3 = ( (wv * 5) + (wv2 * 2) ) / 10

    wv3 = librosa.resample(wv3,44100,sr,scale=True)

    #sd.play(wv3,samplerate=sr)
    if target!=None: sf.write(target,wv3,sr)


In [51]:
class Frame(object):
    """Represents a "frame" of audio data."""
    def __init__(self, bytes, timestamp, duration):
        self.bytes = bytes
        self.timestamp = timestamp
        self.duration = duration

def frame_generator(frame_duration_ms, audio, sample_rate):
    """Generates audio frames from PCM audio data.

    Takes the desired frame duration in milliseconds, the PCM data, and
    the sample rate.

    Yields Frames of the requested duration.
    """
    n = int(sample_rate * (frame_duration_ms / 1000.0) * 2)
    offset = 0
    timestamp = 0.0
    duration = (float(n) / sample_rate) / 2.0
    while offset + n < len(audio):
        yield Frame(audio[offset:offset + n], timestamp, duration)
        timestamp += duration
        offset += n

In [52]:
def gennoise(WORKPATH=WORKPATH,totalfile=1):
    """ 
    Gunakan generate ini untuk membuat SET dari yang ingin di training atau
    untuk SET TESTING, gunakan frame untuk menandakan panjang dari frame
    yang ingin di deteksi
    """
    files=[]
    #Generate AUDIO FILE

    #create the dir 
    for i in WORD_TO_TRAIN:
        try:
            os.mkdir(PATH_TO_TRAIN+"/"+i)
            os.mkdir(PATH_TO_TEST+"/"+i)
        except:
            pass
        
    
    for i in WORD_TO_TRAIN:
        #gunakan PATH_TO_TEST atau PATH_TO_TRAIN
        workdir = WORKPATH+"/"+i+"/"
        filestonoise = glob.glob(workdir+"*.wav")
        for tn in range(totalfile):
            for f in filestonoise:
                rann = str(random.randint(0,5000))
                t_file=f.split('.wav')[0]+'-'+FILE_PREFIX+"-"+rann+".wav"
                
                add_random_noise(f,t_file)
                print(t_file)
    
            print("Selesai proses merekam ke:",workdir)
    print("TELAH SELESAI SEMUA... TERIMA KASIH")
    return files


In [53]:
WORKPATH=PATH_TO_TRAIN
gennoise(WORKPATH,totalfile=10)

./DeadSimpleSpeechRecognizer/newdata/train/cipi\cipi-irza-1240-ns-450.wav
./DeadSimpleSpeechRecognizer/newdata/train/cipi\cipi-irza-2009-ns-1108.wav
./DeadSimpleSpeechRecognizer/newdata/train/cipi\cipi-irza-2227-ns-1999.wav
./DeadSimpleSpeechRecognizer/newdata/train/cipi\cipi-irza-2367-ns-1337.wav
./DeadSimpleSpeechRecognizer/newdata/train/cipi\cipi-irza-255-ns-2250.wav
./DeadSimpleSpeechRecognizer/newdata/train/cipi\cipi-irza-2766-ns-2742.wav
./DeadSimpleSpeechRecognizer/newdata/train/cipi\cipi-irza-2993-ns-601.wav
./DeadSimpleSpeechRecognizer/newdata/train/cipi\cipi-irza-3387-ns-3315.wav
./DeadSimpleSpeechRecognizer/newdata/train/cipi\cipi-irza-366-ns-2454.wav
./DeadSimpleSpeechRecognizer/newdata/train/cipi\cipi-irza-3721-ns-2761.wav
./DeadSimpleSpeechRecognizer/newdata/train/cipi\cipi-irza-4262-ns-1862.wav
./DeadSimpleSpeechRecognizer/newdata/train/cipi\cipi-irza-4793-ns-922.wav
./DeadSimpleSpeechRecognizer/newdata/train/cipi\cipi-irza-518-ns-2161.wav
Selesai proses merekam ke: ./De

./DeadSimpleSpeechRecognizer/newdata/train/cipi\cipi-irza-518-ns-4297.wav
Selesai proses merekam ke: ./DeadSimpleSpeechRecognizer/newdata/train/cipi/
./DeadSimpleSpeechRecognizer/newdata/train/cipi\cipi-irza-1240-ns-682.wav
./DeadSimpleSpeechRecognizer/newdata/train/cipi\cipi-irza-2009-ns-3028.wav
./DeadSimpleSpeechRecognizer/newdata/train/cipi\cipi-irza-2227-ns-2145.wav
./DeadSimpleSpeechRecognizer/newdata/train/cipi\cipi-irza-2367-ns-2902.wav
./DeadSimpleSpeechRecognizer/newdata/train/cipi\cipi-irza-255-ns-165.wav
./DeadSimpleSpeechRecognizer/newdata/train/cipi\cipi-irza-2766-ns-2412.wav
./DeadSimpleSpeechRecognizer/newdata/train/cipi\cipi-irza-2993-ns-1544.wav
./DeadSimpleSpeechRecognizer/newdata/train/cipi\cipi-irza-3387-ns-1231.wav
./DeadSimpleSpeechRecognizer/newdata/train/cipi\cipi-irza-366-ns-2815.wav
./DeadSimpleSpeechRecognizer/newdata/train/cipi\cipi-irza-3721-ns-2215.wav
./DeadSimpleSpeechRecognizer/newdata/train/cipi\cipi-irza-4262-ns-4534.wav
./DeadSimpleSpeechRecognizer/

./DeadSimpleSpeechRecognizer/newdata/train/bangun\bangun-irza-2009-ns-2332.wav
./DeadSimpleSpeechRecognizer/newdata/train/bangun\bangun-irza-2227-ns-4015.wav
./DeadSimpleSpeechRecognizer/newdata/train/bangun\bangun-irza-2367-ns-4795.wav
./DeadSimpleSpeechRecognizer/newdata/train/bangun\bangun-irza-255-ns-3878.wav
./DeadSimpleSpeechRecognizer/newdata/train/bangun\bangun-irza-2766-ns-1877.wav
./DeadSimpleSpeechRecognizer/newdata/train/bangun\bangun-irza-2993-ns-1024.wav
./DeadSimpleSpeechRecognizer/newdata/train/bangun\bangun-irza-366-ns-2408.wav
./DeadSimpleSpeechRecognizer/newdata/train/bangun\bangun-irza-3721-ns-3408.wav
./DeadSimpleSpeechRecognizer/newdata/train/bangun\bangun-irza-3956-ns-2920.wav
./DeadSimpleSpeechRecognizer/newdata/train/bangun\bangun-irza-4262-ns-1648.wav
./DeadSimpleSpeechRecognizer/newdata/train/bangun\bangun-irza-4293-ns-1232.wav
./DeadSimpleSpeechRecognizer/newdata/train/bangun\bangun-irza-4793-ns-227.wav
./DeadSimpleSpeechRecognizer/newdata/train/bangun\bangu

[]

In [40]:
WORKPATH=PATH_TO_TRAIN
import glob
a = glob.glob(WORKPATH+"/*/*.wav")
print(WORKPATH)

/word/train


In [48]:
import pandas as pd
import csv


with open("/home/irq/python/DeepSpeech/data/tessr/test/test.csv", "w") as csvfile:
    wr = csv.writer(csvfile)
    wr.writerow(["wav_filename","wav_filesize","transcript"])
    for i in a:
        wr.writerow([i,os.path.getsize(i),i.split("/")[-2]])   